# 04 - 高级特性

## 学习目标

- 掌握用户自定义函数（UDF）的编写和使用
- 了解复杂数据类型（List、Struct）的操作
- 深入理解延迟执行和查询计划
- 对比不同数据格式的读取性能
- 对比 Daft 与 Pandas 的性能差异

In [ ]:
import io
import sys
import time

import daft
from daft import col
import pandas as pd

df = daft.read_parquet("../data/products.parquet")

## 1. 用户自定义函数（UDF）

Daft 使用 `@daft.func` 装饰器定义逐元素 UDF。通过 Python 类型注解自动推断返回类型，无需手动指定 `return_dtype`。

### 1.1 价格分类 UDF

In [ ]:
# 定义价格分类 UDF（新语法：通过类型注解自动推断返回类型）
@daft.func
def categorize_price(price: float) -> str:
    """价格分类：低(<100)、中(100-1000)、高(>1000)"""
    if price is None:
        return "未知"
    elif price < 100:
        return "低"
    elif price <= 1000:
        return "中"
    else:
        return "高"

# 应用 UDF
df_with_cat = df.with_column("price_category", categorize_price(col("price")))
df_with_cat.select("name", "price", "price_category").show(10)

In [ ]:
# 统计各价格区间的产品数量
df_with_cat.groupby("price_category").agg(
    col("product_id").count().alias("count"),
    col("price").mean().alias("avg_price"),
).show()

### 1.2 文本长度 UDF

In [ ]:
# 定义文本长度 UDF（新语法）
@daft.func
def text_length(text: str) -> int:
    """文本长度"""
    if text is None:
        return 0
    return len(text)

# 应用文本长度 UDF
df_with_len = df.with_column("name_length", text_length(col("name")))
df_with_len.select("name", "name_length").show(10)

### 1.3 自定义 UDF 示例

你可以直接在 Notebook 中定义 UDF。

In [ ]:
# 自定义 UDF：判断是否为高性价比产品（评分高且价格低）
@daft.func
def is_good_value(price: float, rating: float) -> bool:
    if price is None or rating is None:
        return False
    return price < 500 and rating >= 4.0

df_value = df.with_column(
    "good_value",
    is_good_value(col("price"), col("rating")),
)

# 查看高性价比产品
df_value.where(col("good_value") == True).select(
    "name", "price", "rating", "good_value"
).show(10)

## 2. 复杂数据类型

### 2.1 List 类型

In [ ]:
# 创建包含 List 类型列的 DataFrame
df_list = daft.from_pydict({
    "product": ["手机", "电脑", "耳机"],
    "tags": [["电子", "通讯", "智能"], ["电子", "办公"], ["电子", "音频", "便携", "无线"]],
})

print("Schema:")
print(df_list.schema())
df_list.show()

### 2.2 Struct 类型

In [ ]:
# 创建包含 Struct 类型列的 DataFrame
df_struct = daft.from_pydict({
    "product": ["手机", "电脑"],
    "specs": [
        {"cpu": "A16", "ram": 8, "storage": 256},
        {"cpu": "M2", "ram": 16, "storage": 512},
    ],
})

print("Schema:")
print(df_struct.schema())
df_struct.show()

## 3. 延迟执行和查询计划

### 3.1 查看查询计划

`explain(show_all=True)` 展示未优化逻辑计划、优化后逻辑计划和物理计划。

In [ ]:
# 构建一个复杂查询
query = (
    df
    .where(col("price") > 1000)
    .select("product_id", "name", "category", "price", "rating")
    .where(col("rating").not_null())
    .sort("price", desc=True)
    .limit(20)
)

# 查看完整查询计划
old_stdout = sys.stdout
sys.stdout = buffer = io.StringIO()
try:
    query.explain(show_all=True)
finally:
    sys.stdout = old_stdout
plan = buffer.getvalue()
print(plan)

### 3.2 查询优化原理

Daft 的查询优化器会自动进行以下优化：

- **谓词下推（Predicate Pushdown）**：将过滤条件尽可能推到数据源附近
- **列裁剪（Column Pruning）**：只读取需要的列
- **投影合并（Projection Merging）**：合并连续的 select 操作

对比未优化和优化后的计划，可以看到优化器的效果。

In [ ]:
# 简单查询 vs 复杂查询的计划对比
simple_query = df.select("name", "price").where(col("price") > 100)

old_stdout = sys.stdout
sys.stdout = buffer = io.StringIO()
try:
    simple_query.explain(show_all=True)
finally:
    sys.stdout = old_stdout

print("=== 简单查询计划 ===")
print(buffer.getvalue())

## 4. 性能对比

### 4.1 CSV vs Parquet vs JSON 读取性能

In [ ]:
# 对比三种格式的读取性能
results = {}
for fmt, path in [
    ("csv", "../data/products.csv"),
    ("parquet", "../data/products.parquet"),
    ("json", "../data/products.json"),
]:
    reader = {"csv": daft.read_csv, "parquet": daft.read_parquet, "json": daft.read_json}[fmt]
    start = time.time()
    reader(path).collect()
    results[fmt] = round(time.time() - start, 4)

print("读取 + collect 耗时（秒）:")
for fmt, t in sorted(results.items(), key=lambda x: x[1]):
    print(f"  {fmt:>8s}: {t:.4f}s")

fastest = min(results, key=results.get)
print(f"\n最快格式: {fastest}")

### 4.2 Daft vs Pandas

In [ ]:
# 对比 Daft 和 Pandas 读取 CSV 的性能
csv_path = "../data/products.csv"

# Pandas
start = time.time()
pd.read_csv(csv_path)
pandas_time = round(time.time() - start, 4)

# Daft
start = time.time()
daft.read_csv(csv_path).collect()
daft_time = round(time.time() - start, 4)

print("CSV 读取耗时（秒）:")
print(f"  Pandas: {pandas_time:.4f}s")
print(f"  Daft:   {daft_time:.4f}s")

if daft_time < pandas_time:
    speedup = pandas_time / daft_time
    print(f"\nDaft 快 {speedup:.1f}x")
else:
    speedup = daft_time / pandas_time
    print(f"\nPandas 快 {speedup:.1f}x（小数据集下 Pandas 可能更快）")

## 总结

本节学习了 Daft 的高级特性：

| 特性 | 说明 |
|------|------|
| `@daft.func` UDF | 逐元素自定义函数，类型注解自动推断返回类型 |
| `@daft.func.batch` | 批处理 UDF，接收 `daft.Series`，性能更高 |
| `@daft.cls` | 有状态 UDF（如加载 ML 模型），初始化一次复用多行 |
| List 类型 | 列表类型列，支持嵌套数据 |
| Struct 类型 | 结构体类型列，支持字段访问 |
| `explain()` | 查看查询计划，理解优化效果 |
| 性能对比 | Parquet 通常是最快的格式 |

## 练习题

1. 编写一个 UDF，根据 `review_count` 和 `rating` 计算产品热度分数
2. 构建一个包含多步操作的查询，查看优化前后的查询计划差异
3. 生成更大的数据集（10 万条），重新对比 Daft 和 Pandas 的性能

## 下一步

继续学习 [05_ai_multimodal.ipynb](./05_ai_multimodal.ipynb) —— 掌握 Daft 的 AI Functions 和多模态能力。